In [1]:
from pyspark.sql import SparkSession
import pandas as pd

# 다른 필요한 imports도 함께
from pyspark.sql.functions import col, min as spark_min, max as spark_max
import glob
import os
import time
import statistics

# Spark + Iceberg + Avro 패키지 설정
spark = SparkSession.builder \
    .appName("IcebergWithAvro") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.4.0,org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.4.3") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "hadoop") \
    .config("spark.sql.catalog.demo.warehouse", "file:///home/jovyan/data/warehouse") \
    .getOrCreate()

print("✅ Spark + Iceberg 설정 완료!")

# 카탈로그 확인
spark.sql("SHOW CATALOGS").show()

✅ Spark + Iceberg 설정 완료!
+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [2]:
# =============================================================================
# 📁 Data Layer 완전 분석 - 물리적 파일 구조만 집중
# =============================================================================

print("📁 Data Layer 완전 분석 - 물리적 파일 구조")
print("=" * 50)
print("🎯 분석 대상: 실제 데이터가 저장되는 물리적 파일들")
print("  📄 Data Files (.parquet)")
print("  🗑️ Delete Files (.parquet)")
print("  📁 파티션 디렉토리 구조")
print("  💾 파일 크기 및 분포")
print("=" * 50)

mor_table_path = "/home/jovyan/data/warehouse/blog/orders_mor"

# 1. 물리적 파일 구조 탐색
print("🗂️ 1. 물리적 파일 구조 분석:")
print("=" * 30)

# data 디렉토리만 분석 (metadata 제외)
data_path = f"{mor_table_path}/data"

def analyze_data_directory(path, prefix=""):
    """데이터 디렉토리만 분석"""
    items = []
    try:
        for item in sorted(os.listdir(path)):
            if item.startswith('.'):
                continue
            item_path = os.path.join(path, item)
            
            if os.path.isfile(item_path):
                size = os.path.getsize(item_path)
                
                # 파일 타입만 구분 (데이터 레이어)
                if item.endswith('.parquet'):
                    if 'delete' in item.lower():
                        file_type = "🗑️ DELETE FILE"
                    else:
                        file_type = "📄 DATA FILE"
                else:
                    file_type = "📄 OTHER FILE"
                
                items.append(f"{prefix}├── {item} ({size:,} bytes) {file_type}")
            elif os.path.isdir(item_path):
                items.append(f"{prefix}├── 📁 {item}/")
                items.extend(analyze_data_directory(item_path, prefix + "│   "))
    except:
        pass
    return items

if os.path.exists(data_path):
    print("📁 /data 디렉토리 구조:")
    structure = analyze_data_directory(data_path)
    for item in structure:
        print(item)
else:
    print("❌ /data 디렉토리가 없습니다.")

# 2. 파티션 구조 분석
print(f"\n🗂️ 2. 파티션 구조 분석:")
print("=" * 30)

partition_dirs = glob.glob(f"{data_path}/order_date=*") if os.path.exists(data_path) else []
print(f"파티션 개수: {len(partition_dirs)}")

for partition_dir in sorted(partition_dirs):
    partition_name = os.path.basename(partition_dir)
    
    # 이 파티션의 모든 파일
    partition_files = glob.glob(f"{partition_dir}/*.parquet")
    data_files_in_partition = [f for f in partition_files if 'delete' not in os.path.basename(f).lower()]
    delete_files_in_partition = [f for f in partition_files if 'delete' in os.path.basename(f).lower()]
    
    total_size = sum(os.path.getsize(f) for f in partition_files)
    
    print(f"\n📁 {partition_name}:")
    print(f"  📄 데이터 파일: {len(data_files_in_partition)}개")
    print(f"  🗑️ Delete 파일: {len(delete_files_in_partition)}개")
    print(f"  💾 총 크기: {total_size:,} bytes ({total_size/1024:.1f} KB)")

# 3. 개별 데이터 파일 분석
print(f"\n📄 3. 개별 데이터 파일 분석:")
print("=" * 30)

all_data_files = glob.glob(f"{data_path}/**/*.parquet", recursive=True) if os.path.exists(data_path) else []
data_only_files = [f for f in all_data_files if 'delete' not in os.path.basename(f).lower()]

print(f"총 데이터 파일: {len(data_only_files)}개")

for i, file_path in enumerate(data_only_files[:5]):  # 처음 5개만
    file_size = os.path.getsize(file_path)
    relative_path = file_path.replace(mor_table_path, "")
    
    print(f"\n📄 파일 #{i+1}: {relative_path}")
    print(f"  💾 크기: {file_size:,} bytes ({file_size/1024:.1f} KB)")
    
    try:
        # 파일 내용 분석 (물리적 특성만)
        df = spark.read.parquet(file_path)
        record_count = df.count()
        
        print(f"  📊 레코드 수: {record_count:,}개")
        print(f"  📏 컬럼 수: {len(df.columns)}개")
        
        if record_count > 0:
            # 날짜 범위 (파티셔닝 확인용)
            from pyspark.sql.functions import min as spark_min, max as spark_max
            date_range = df.agg(
                spark_min("order_date").alias("min_date"),
                spark_max("order_date").alias("max_date")
            ).collect()[0]
            
            print(f"  📅 날짜 범위: {date_range['min_date']} ~ {date_range['max_date']}")
            
            # 압축률 계산 (대략)
            estimated_raw_size = record_count * 100  # 대략 100bytes per record
            compression_ratio = estimated_raw_size / file_size if file_size > 0 else 0
            print(f"  🗜️ 압축률: {compression_ratio:.1f}:1 (추정)")
            
    except Exception as e:
        print(f"  ❌ 분석 실패: {e}")

# 4. Delete Files 물리적 분석
print(f"\n🗑️ 4. Delete Files 물리적 분석:")
print("=" * 30)

delete_only_files = [f for f in all_data_files if 'delete' in os.path.basename(f).lower()]
print(f"총 Delete 파일: {len(delete_only_files)}개")

for i, delete_file in enumerate(delete_only_files):
    file_size = os.path.getsize(delete_file)
    relative_path = delete_file.replace(mor_table_path, "")
    
    print(f"\n🗑️ Delete 파일 #{i+1}: {relative_path}")
    print(f"  💾 크기: {file_size:,} bytes ({file_size/1024:.1f} KB)")
    
    try:
        delete_df = spark.read.parquet(delete_file)
        delete_count = delete_df.count()
        
        print(f"  📊 삭제 레코드 수: {delete_count:,}개")
        
        # Delete file 물리적 특성
        if delete_count > 0:
            columns = delete_df.columns
            print(f"  📏 컬럼들: {columns}")
            
            # Positional vs Equality 구분
            if 'file_path' in columns and 'pos' in columns:
                print(f"  📍 타입: Positional Delete (파일+위치 기반)")
            else:
                print(f"  🟰 타입: Equality Delete (값 기반)")
                
    except Exception as e:
        print(f"  ❌ 분석 실패: {e}")

# 5. 파일 크기 분포 분석
print(f"\n📊 5. 파일 크기 분포 분석:")
print("=" * 30)

if data_only_files:
    file_sizes = [os.path.getsize(f) for f in data_only_files]
    total_size = sum(file_sizes)
    
    print(f"📊 데이터 파일 통계:")
    print(f"  총 크기: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")
    print(f"  평균 크기: {statistics.mean(file_sizes):,.0f} bytes")
    print(f"  최소 크기: {min(file_sizes):,} bytes")
    print(f"  최대 크기: {max(file_sizes):,} bytes")
    print(f"  중간값: {statistics.median(file_sizes):,.0f} bytes")
    
    # 크기별 분류
    small_files = [s for s in file_sizes if s < 100 * 1024]  # < 100KB
    medium_files = [s for s in file_sizes if 100 * 1024 <= s < 1024 * 1024]  # 100KB-1MB
    large_files = [s for s in file_sizes if s >= 1024 * 1024]  # >= 1MB
    
    print(f"\n📏 크기별 분포:")
    print(f"  작은 파일 (< 100KB): {len(small_files)}개")
    print(f"  중간 파일 (100KB-1MB): {len(medium_files)}개")
    print(f"  큰 파일 (>= 1MB): {len(large_files)}개")

print(f"\n✅ Data Layer 물리적 분석 완료!")
print("👉 다음: Metadata Layer 논리적 분석")

📁 Data Layer 완전 분석 - 물리적 파일 구조
🎯 분석 대상: 실제 데이터가 저장되는 물리적 파일들
  📄 Data Files (.parquet)
  🗑️ Delete Files (.parquet)
  📁 파티션 디렉토리 구조
  💾 파일 크기 및 분포
🗂️ 1. 물리적 파일 구조 분석:
📁 /data 디렉토리 구조:
├── 📁 order_date=2023-01-01/
│   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet (1,834 bytes) 📄 DATA FILE
├── 📁 order_date=2023-01-02/
│   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00002.parquet (1,865 bytes) 📄 DATA FILE
├── 📁 order_date=2023-01-03/
│   ├── 00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001-deletes.parquet (1,595 bytes) 🗑️ DELETE FILE
│   ├── 00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001.parquet (1,806 bytes) 📄 DATA FILE
│   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00003.parquet (1,814 bytes) 📄 DATA FILE
├── 📁 order_date=2023-01-05/
│   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00004.parquet (1,835 bytes) 📄 DATA FILE
├── 📁 order_date=2023-01-06/
│   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00005.parquet (1,813 bytes) 📄 DATA FILE
├── 📁 o

In [3]:
# =============================================================================
# 🗂️ Metadata Layer 완전 분석 - 논리적 구조만 집중
# =============================================================================

print("🗂️ Metadata Layer 완전 분석 - 논리적 구조")
print("=" * 50)
print("🎯 분석 대상: 테이블 메타데이터와 관리 정보")
print("  📊 Metadata Files (.json) - 테이블 정의")
print("  📋 Manifest Files (.avro) - 파일 목록") 
print("  📄 Manifest Lists (.avro) - 매니페스트 목록")
print("  🌳 메타데이터 계층 구조")
print("=" * 50)

metadata_path = f"{mor_table_path}/metadata"

# 1. 메타데이터 파일 분류
print("📂 1. 메타데이터 파일 분류:")
print("=" * 30)

metadata_files = {
    'metadata_files': [],
    'manifest_files': [],
    'manifest_lists': [],
    'other_files': []
}

for file in os.listdir(metadata_path):
    file_path = os.path.join(metadata_path, file)
    size = os.path.getsize(file_path)
    
    if file.endswith('.metadata.json'):
        metadata_files['metadata_files'].append((file, size))
    elif file.endswith('.avro') and not file.startswith('snap-'):
        metadata_files['manifest_files'].append((file, size))
    elif file.startswith('snap-') and file.endswith('.avro'):
        metadata_files['manifest_lists'].append((file, size))
    else:
        metadata_files['other_files'].append((file, size))

for category, files in metadata_files.items():
    category_name = category.replace('_', ' ').title()
    print(f"📊 {category_name}: {len(files)}개")
    for filename, size in files:
        print(f"  📄 {filename} ({size:,} bytes)")

# 2. Metadata Files (.json) 논리적 분석
print(f"\n📊 2. Metadata Files 논리적 구조:")
print("=" * 30)

latest_metadata = max(metadata_files['metadata_files'], key=lambda x: x[1])[0]
metadata_file_path = os.path.join(metadata_path, latest_metadata)

import json
with open(metadata_file_path, 'r') as f:
    metadata = json.load(f)

print(f"📄 최신 메타데이터: {latest_metadata}")
print(f"📊 포맷 버전: {metadata['format-version']}")
print(f"🆔 테이블 UUID: {metadata['table-uuid']}")
print(f"📍 테이블 위치: {metadata['location']}")

# 스키마 진화 분석
schemas = metadata.get('schemas', [])
current_schema_id = metadata.get('current-schema-id')

print(f"\n📋 스키마 진화 히스토리:")
for schema in schemas:
    schema_id = schema['schema-id']
    is_current = schema_id == current_schema_id
    field_count = len(schema['fields'])
    
    print(f"  스키마 ID {schema_id} ({'현재' if is_current else '이전'}):")
    print(f"    컬럼 수: {field_count}개")
    
    if is_current:
        print(f"    컬럼들:")
        for field in schema['fields']:
            required = '(필수)' if field['required'] else '(선택)'
            print(f"      - {field['name']}: {field['type']} {required}")

# 스냅샷 분석
snapshots = metadata.get('snapshots', [])
print(f"\n📸 스냅샷 히스토리 ({len(snapshots)}개):")
for i, snapshot in enumerate(snapshots):
    snapshot_id = snapshot['snapshot-id']
    operation = snapshot.get('operation', 'unknown')
    timestamp = snapshot.get('timestamp-ms', 0)
    summary = snapshot.get('summary', {})
    
    print(f"  #{i+1} ID: {snapshot_id}")
    print(f"      작업: {operation}")
    print(f"      시간: {timestamp}")
    
    # MOR 관련 정보
    added_files = summary.get('added-data-files', '0')
    deleted_files = summary.get('deleted-data-files', '0')
    added_delete_files = summary.get('added-delete-files', '0')
    
    print(f"      파일 변화: +{added_files} data, -{deleted_files} data, +{added_delete_files} delete")

# 3. Manifest Files (.avro) 논리적 분석
print(f"\n📋 3. Manifest Files 논리적 구조:")
print("=" * 30)

for filename, size in metadata_files['manifest_files'][:3]:  # 처음 3개만
    file_path = os.path.join(metadata_path, filename)
    
    print(f"\n📄 {filename}:")
    print(f"  크기: {size:,} bytes")
    
    try:
        manifest_df = spark.read.format("avro").load(f"file://{file_path}")
        entry_count = manifest_df.count()
        
        print(f"  항목 수: {entry_count}개")
        
        if entry_count > 0:
            # 상태별 분포
            print("  📊 파일 상태:")
            status_dist = manifest_df.groupBy("status").count().collect()
            for row in status_dist:
                status_text = {1: "추가됨", 2: "삭제됨", 0: "기존"}.get(row['status'], "기타")
                print(f"    {status_text}: {row['count']}개")
            
            # 스냅샷별 분포  
            print("  📸 스냅샷별 분포:")
            snapshot_dist = manifest_df.groupBy("snapshot_id").count().collect()
            for row in snapshot_dist[:3]:  # 최대 3개만
                print(f"    스냅샷 {row['snapshot_id']}: {row['count']}개")
                
    except Exception as e:
        print(f"  ❌ 분석 실패: {e}")

# 4. Manifest Lists 논리적 분석
print(f"\n📄 4. Manifest Lists 논리적 구조:")
print("=" * 30)

for filename, size in metadata_files['manifest_lists']:
    file_path = os.path.join(metadata_path, filename)
    
    print(f"\n📄 {filename}:")
    print(f"  크기: {size:,} bytes")
    
    try:
        manifest_list_df = spark.read.format("avro").load(f"file://{file_path}")
        manifest_count = manifest_list_df.count()
        
        print(f"  포함된 Manifest: {manifest_count}개")
        
        if manifest_count > 0:
            print("  📋 Manifest 목록:")
            manifest_list_df.select("manifest_path", "manifest_length").show(5, truncate=False)
            
    except Exception as e:
        print(f"  ❌ 분석 실패: {e}")

# 5. 메타데이터 트리 구조 시각화
print(f"\n🌳 5. 메타데이터 계층 구조:")
print("=" * 30)

print("""
Iceberg Metadata 3단계 구조:

📊 Level 1: Metadata Files (.json)
├── 테이블 스키마 정의
├── 파티션 스펙 설정  
├── 스냅샷 목록 관리
└── 전체 테이블 상태
    ↓ 참조
📄 Level 2: Manifest Lists (.avro)  
├── 스냅샷별 Manifest 그룹
├── 파티션별 Manifest 분류
└── Manifest 위치 정보
    ↓ 참조  
📋 Level 3: Manifest Files (.avro)
├── 개별 데이터 파일 목록
├── 파일별 통계 정보
├── 컬럼 경계값 (min/max)
└── 삭제 상태 추적
    ↓ 가리킴
📄 Data Layer: Actual Files (.parquet)
""")

print(f"\n✅ Metadata Layer 논리적 분석 완료!")
print("👉 이제 두 레이어가 명확히 분리되었습니다!")

🗂️ Metadata Layer 완전 분석 - 논리적 구조
🎯 분석 대상: 테이블 메타데이터와 관리 정보
  📊 Metadata Files (.json) - 테이블 정의
  📋 Manifest Files (.avro) - 파일 목록
  📄 Manifest Lists (.avro) - 매니페스트 목록
  🌳 메타데이터 계층 구조
📂 1. 메타데이터 파일 분류:
📊 Metadata Files: 12개
  📄 v1.metadata.json (1,677 bytes)
  📄 v10.metadata.json (11,412 bytes)
  📄 v11.metadata.json (12,665 bytes)
  📄 v12.metadata.json (13,660 bytes)
  📄 v2.metadata.json (2,741 bytes)
  📄 v3.metadata.json (3,807 bytes)
  📄 v4.metadata.json (4,936 bytes)
  📄 v5.metadata.json (5,997 bytes)
  📄 v6.metadata.json (7,001 bytes)
  📄 v7.metadata.json (7,909 bytes)
  📄 v8.metadata.json (9,161 bytes)
  📄 v9.metadata.json (10,160 bytes)
📊 Manifest Files: 8개
  📄 1402a55e-3b81-4e6f-afb1-338c510fc079-m0.avro (10,910 bytes)
  📄 18f20e54-9d72-4e4f-b8fe-766e6ff40279-m0.avro (7,330 bytes)
  📄 20ce2ed5-b000-4bb7-85e5-5cec65827504-m0.avro (7,686 bytes)
  📄 7b4a9d3d-cec2-46eb-915d-e975a16a860c-m0.avro (9,927 bytes)
  📄 a902da7e-5da3-4e7d-a892-af8bb1a0d4be-m0.avro (7,306 bytes)
  📄 e66faff5

In [4]:
# Iceberg 시스템 테이블로 Manifest 내용 간접 확인
print("📊 Iceberg 시스템 테이블로 Manifest 내용 확인:")
print("=" * 50)

try:
    print("📄 1. 데이터 파일 정보 (Manifest 내용):")
    files_df = spark.sql("""
        SELECT 
            file_path,
            file_format, 
            record_count,
            file_size_in_bytes,
            value_counts,
            null_value_counts,
            lower_bounds,
            upper_bounds
        FROM demo.blog.orders_mor.files 
        LIMIT 3
    """)
    
    files_df.show(truncate=False, vertical=True)
    
    print("\n🗑️ 2. Delete 파일 정보:")
    try:
        spark.sql("SELECT * FROM demo.blog.orders_mor.delete_files LIMIT 3").show(truncate=False)
    except:
        print("  ❌ Delete files 시스템 테이블 없음")
    
    print("\n📋 3. Manifest 파일 정보:")
    manifests_df = spark.sql("""
        SELECT 
            path,
            length,
            partition_spec_id,
            added_snapshot_id,
            added_data_files_count,
            existing_data_files_count,
            deleted_data_files_count
        FROM demo.blog.orders_mor.manifests
        LIMIT 3
    """)
    
    manifests_df.show(truncate=False, vertical=True)
    
    print("\n📸 4. 스냅샷별 Manifest 변화:")
    spark.sql("""
        SELECT 
            snapshot_id,
            operation,
            summary['added-data-files'] as added_files,
            summary['deleted-data-files'] as deleted_files,  
            summary['added-delete-files'] as added_delete_files,
            summary['total-data-files'] as total_files
        FROM demo.blog.orders_mor.snapshots
        ORDER BY committed_at DESC
        LIMIT 5
    """).show()
    
except Exception as e:
    print(f"❌ 시스템 테이블 조회 실패: {e}")
    print("👉 기본 테이블 정보로 대체:")
    
    try:
        spark.sql("DESCRIBE EXTENDED demo.blog.orders_mor").show(truncate=False)
    except Exception as e2:
        print(f"❌ 기본 조회도 실패: {e2}")

print(f"\n💡 Manifest 파일 내용 요약:")
print("=" * 30)
print("📄 Manifest Files (.avro) 내용:")
print("  - status: 파일 상태 (1=추가, 2=삭제)")
print("  - snapshot_id: 스냅샷 ID")
print("  - data_file: 데이터 파일 정보")
print("    ├─ file_path: 파일 경로")
print("    ├─ file_format: 파일 형식 (PARQUET)")
print("    ├─ record_count: 레코드 수")
print("    ├─ file_size_in_bytes: 파일 크기")
print("    ├─ value_counts: 컬럼별 값 개수")
print("    ├─ null_value_counts: NULL 값 개수")
print("    ├─ lower_bounds: 컬럼 최솟값")
print("    └─ upper_bounds: 컬럼 최댓값")

print("\n📄 Manifest Lists (.avro) 내용:")
print("  - manifest_path: Manifest 파일 경로")
print("  - manifest_length: Manifest 파일 크기")
print("  - partition_spec_id: 파티션 스펙 ID")
print("  - added_snapshot_id: 추가된 스냅샷 ID")
print("  - added_data_files_count: 추가된 데이터 파일 수")
print("  - existing_data_files_count: 기존 데이터 파일 수")
print("  - deleted_data_files_count: 삭제된 데이터 파일 수")

📊 Iceberg 시스템 테이블로 Manifest 내용 확인:
📄 1. 데이터 파일 정보 (Manifest 내용):
-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------
 file_path          | file:/home/jovyan/data/warehouse/blog/orders_mor/data/order_date=2023-01-01/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet  
 file_format        | PARQUET                                                                                                                                 
 record_count       | 1                                                                                                                                       
 file_size_in_bytes | 1834                                                                                                                                    
 value_counts       | {1 -> 1, 2 -> 1, 3 -> 1, 4 -> 1, 5 -> 1, 6 -> 1}                                                                      

In [6]:
# 바이너리 레벨에서 Avro 구조 분석
def analyze_avro_structure(file_path):
    """Avro 파일의 바이너리 구조 분석"""
    try:
        with open(file_path, 'rb') as f:
            # Avro 매직 넘버 확인
            magic = f.read(4)
            print(f"  🔮 매직 넘버: {magic}")
            
            if magic == b'Obj\x01':
                print("  ✅ 유효한 Avro 파일")
                
                # 메타데이터 길이 읽기
                f.seek(4)
                meta_length_bytes = f.read(8)
                
                # 전체 파일 크기
                f.seek(0, 2)  # 파일 끝으로
                file_size = f.tell()
                
                print(f"  📏 파일 크기: {file_size:,} bytes")
                
                # 처음 100바이트 헥스 덤프
                f.seek(0)
                first_100 = f.read(100)
                print(f"  🔍 처음 100바이트 (헥스):")
                hex_str = ' '.join(f'{b:02x}' for b in first_100)
                for i in range(0, len(hex_str), 48):
                    print(f"    {hex_str[i:i+48]}")
                
                # ASCII 문자열 찾기
                f.seek(0)
                content = f.read(min(1000, file_size))
                ascii_strings = []
                current_string = ""
                
                for byte in content:
                    if 32 <= byte <= 126:  # 출력 가능한 ASCII
                        current_string += chr(byte)
                    else:
                        if len(current_string) >= 4:
                            ascii_strings.append(current_string)
                        current_string = ""
                
                if ascii_strings:
                    print(f"  📝 발견된 문자열들:")
                    for s in ascii_strings[:5]:
                        print(f"    '{s}'")
                
            else:
                print("  ❌ Avro 형식이 아님")
                
    except Exception as e:
        print(f"  ❌ 분석 실패: {e}")

# Manifest 파일들 분석
metadata_path = f"{mor_table_path}/metadata"
manifest_files = [f for f in os.listdir(metadata_path) if f.endswith('.avro') and not f.startswith('snap-')]
manifest_lists = [f for f in os.listdir(metadata_path) if f.startswith('snap-') and f.endswith('.avro')]

print("🔍 Avro 파일 바이너리 구조 분석:")
print("=" * 40)

# Manifest 파일들 분석
for filename in manifest_files[:2]:
    print(f"\n📄 {filename}:")
    file_path = os.path.join(metadata_path, filename)
    analyze_avro_structure(file_path)

# Manifest List들 분석  
print(f"\n📄 Manifest Lists:")
for filename in manifest_lists[:2]:
    print(f"\n📄 {filename}:")
    file_path = os.path.join(metadata_path, filename)
    analyze_avro_structure(file_path)

🔍 Avro 파일 바이너리 구조 분석:

📄 1402a55e-3b81-4e6f-afb1-338c510fc079-m0.avro:
  🔮 매직 넘버: b'Obj\x01'
  ✅ 유효한 Avro 파일
  📏 파일 크기: 10,910 bytes
  🔍 처음 100바이트 (헥스):
    4f 62 6a 01 10 0c 73 63 68 65 6d 61 cc 0a 7b 22 
    74 79 70 65 22 3a 22 73 74 72 75 63 74 22 2c 22 
    73 63 68 65 6d 61 2d 69 64 22 3a 33 2c 22 66 69 
    65 6c 64 73 22 3a 5b 7b 22 69 64 22 3a 31 2c 22 
    6e 61 6d 65 22 3a 22 6f 72 64 65 72 5f 69 64 22 
    2c 22 72 65 71 75 69 72 65 64 22 3a 66 61 6c 73 
    65 2c 22 74
  📝 발견된 문자열들:
    'schema'
    '{"type":"struct","schema-id":3,"fields":[{"id":1,"name":"order_id","required":false,"type":"long"},{"id":2,"name":"customer_id","required":false,"type":"long"},{"id":3,"name":"product_name","required":false,"type":"string"},{"id":4,"name":"order_date","required":false,"type":"date"},{"id":5,"name":"amount","required":false,"type":"decimal(12, 2)"},{"id":6,"name":"status","required":false,"type":"string"},{"id":7,"name":"discount_rate","required":false,"type":"decimal(5, 2)","d

In [8]:
# avro-tools 명령어 수정해서 실행
import subprocess
import os

metadata_path = f"{mor_table_path}/metadata"
avro_tools_path = "/tmp/avro-tools.jar"

# Manifest 파일들 찾기
manifest_files = [f for f in os.listdir(metadata_path) 
                 if f.endswith('.avro') and not f.startswith('snap-')]
manifest_lists = [f for f in os.listdir(metadata_path) 
                  if f.startswith('snap-') and f.endswith('.avro')]

print("🔍 avro-tools로 실제 Manifest 파일 내용 보기:")
print("=" * 60)

# 1. 첫 번째 Manifest 파일 분석
if manifest_files:
    first_manifest = os.path.join(metadata_path, manifest_files[0])
    
    print(f"\n📄 Manifest File: {manifest_files[0]}")
    print("-" * 50)
    
    # 스키마 먼저 확인
    print("📋 1. 스키마 구조:")
    schema_result = subprocess.run([
        'java', '-jar', avro_tools_path, 'getschema', first_manifest
    ], capture_output=True, text=True)
    
    if schema_result.returncode == 0:
        try:
            import json
            schema_json = json.loads(schema_result.stdout)
            print(json.dumps(schema_json, indent=2, ensure_ascii=False))
        except:
            print(schema_result.stdout)
    
    # 데이터 내용 확인 (올바른 명령어 사용)
    print(f"\n📝 2. 실제 데이터 내용:")
    print("-" * 30)
    
    # tojson 명령어 사용 (pretty 옵션 없이)
    data_result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', first_manifest
    ], capture_output=True, text=True)
    
    if data_result.returncode == 0:
        lines = data_result.stdout.strip().split('\n')
        print(f"총 {len(lines)}개 레코드 발견!")
        
        # 처음 2개 레코드만 예쁘게 출력
        for i, line in enumerate(lines[:2]):
            if line.strip():
                try:
                    record = json.loads(line)
                    print(f"\n📝 레코드 #{i+1}:")
                    print(json.dumps(record, indent=2, ensure_ascii=False))
                    
                    # 의미 해석
                    print(f"\n💡 레코드 #{i+1} 해석:")
                    status = record.get('status', 'unknown')
                    status_text = {1: "추가됨", 2: "삭제됨", 0: "기존"}.get(status, f"상태:{status}")
                    print(f"  📊 파일 상태: {status_text}")
                    
                    if 'data_file' in record:
                        df = record['data_file']
                        print(f"  📄 파일 경로: {df.get('file_path', 'N/A')}")
                        print(f"  📏 파일 크기: {df.get('file_size_in_bytes', 0):,} bytes")
                        print(f"  📊 레코드 수: {df.get('record_count', 0):,}개")
                        
                except json.JSONDecodeError:
                    print(f"JSON 파싱 실패: {line[:100]}...")
    else:
        print(f"❌ 데이터 읽기 실패: {data_result.stderr}")

# 2. 첫 번째 Manifest List 분석
if manifest_lists:
    first_list = os.path.join(metadata_path, manifest_lists[0])
    
    print(f"\n\n📋 Manifest List: {manifest_lists[0]}")
    print("=" * 60)
    
    # 스키마 확인
    print("📋 1. 스키마 구조:")
    schema_result = subprocess.run([
        'java', '-jar', avro_tools_path, 'getschema', first_list
    ], capture_output=True, text=True)
    
    if schema_result.returncode == 0:
        try:
            schema_json = json.loads(schema_result.stdout)
            print(json.dumps(schema_json, indent=2, ensure_ascii=False))
        except:
            print(schema_result.stdout)
    
    # 데이터 내용 확인
    print(f"\n📝 2. 실제 데이터 내용:")
    print("-" * 30)
    
    data_result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', first_list
    ], capture_output=True, text=True)
    
    if data_result.returncode == 0:
        lines = data_result.stdout.strip().split('\n')
        print(f"총 {len(lines)}개 레코드 발견!")
        
        # 처음 2개 레코드만 예쁘게 출력
        for i, line in enumerate(lines[:2]):
            if line.strip():
                try:
                    record = json.loads(line)
                    print(f"\n📝 Manifest 엔트리 #{i+1}:")
                    print(json.dumps(record, indent=2, ensure_ascii=False))
                    
                    # 의미 해석
                    print(f"\n💡 엔트리 #{i+1} 해석:")
                    manifest_path = record.get('manifest_path', 'N/A')
                    print(f"  📄 Manifest 파일: {os.path.basename(manifest_path)}")
                    print(f"  📏 크기: {record.get('manifest_length', 0):,} bytes")
                    print(f"  🗂️ 파티션 스펙: {record.get('partition_spec_id', 'N/A')}")
                    
                    content = record.get('content', 'N/A')
                    content_text = {0: "DATA", 1: "DELETE"}.get(content, f"타입:{content}")
                    print(f"  📊 타입: {content_text}")
                    
                except json.JSONDecodeError:
                    print(f"JSON 파싱 실패: {line[:100]}...")
    else:
        print(f"❌ 데이터 읽기 실패: {data_result.stderr}")

🔍 avro-tools로 실제 Manifest 파일 내용 보기:

📄 Manifest File: 1402a55e-3b81-4e6f-afb1-338c510fc079-m0.avro
--------------------------------------------------
📋 1. 스키마 구조:
{
  "type": "record",
  "name": "manifest_entry",
  "fields": [
    {
      "name": "status",
      "type": "int",
      "field-id": 0
    },
    {
      "name": "snapshot_id",
      "type": [
        "null",
        "long"
      ],
      "default": null,
      "field-id": 1
    },
    {
      "name": "sequence_number",
      "type": [
        "null",
        "long"
      ],
      "default": null,
      "field-id": 3
    },
    {
      "name": "file_sequence_number",
      "type": [
        "null",
        "long"
      ],
      "default": null,
      "field-id": 4
    },
    {
      "name": "data_file",
      "type": {
        "type": "record",
        "name": "r2",
        "fields": [
          {
            "name": "content",
            "type": "int",
            "doc": "Contents of the file: 0=data, 1=position deletes, 2=

In [9]:
# =============================================================================
# 🔍 Manifest & Manifest List 하나씩 Key-Value로 깔끔하게 보기
# =============================================================================

import subprocess
import json
import os

metadata_path = f"{mor_table_path}/metadata"
avro_tools_path = "/tmp/avro-tools.jar"

# 파일들 찾기
manifest_files = [f for f in os.listdir(metadata_path) 
                 if f.endswith('.avro') and not f.startswith('snap-')]
manifest_lists = [f for f in os.listdir(metadata_path) 
                  if f.startswith('snap-') and f.endswith('.avro')]

def pretty_print_kv(title, data, indent=0):
    """Key-Value를 예쁘게 출력"""
    prefix = "  " * indent
    print(f"{prefix}📋 {title}:")
    
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict):
                print(f"{prefix}  🔹 {key}:")
                pretty_print_kv("", value, indent + 2)
            elif isinstance(value, list):
                print(f"{prefix}  🔹 {key}: [{len(value)}개 항목]")
                for i, item in enumerate(value[:3]):  # 처음 3개만
                    if isinstance(item, dict):
                        print(f"{prefix}    [{i}]:")
                        pretty_print_kv("", item, indent + 3)
                    else:
                        print(f"{prefix}    [{i}]: {item}")
                if len(value) > 3:
                    print(f"{prefix}    ... (총 {len(value)}개)")
            else:
                # 값이 너무 길면 줄이기
                if isinstance(value, str) and len(str(value)) > 100:
                    display_value = str(value)[:100] + "..."
                else:
                    display_value = value
                print(f"{prefix}  🔹 {key}: {display_value}")
    else:
        print(f"{prefix}  {data}")

# 1. MANIFEST FILE 하나만 상세 분석
print("📄 MANIFEST FILE 상세 분석")
print("=" * 60)

if manifest_files:
    selected_manifest = manifest_files[0]  # 첫 번째 선택
    manifest_path = os.path.join(metadata_path, selected_manifest)
    
    print(f"🎯 선택된 파일: {selected_manifest}")
    print(f"📏 파일 크기: {os.path.getsize(manifest_path):,} bytes")
    
    # avro-tools로 데이터 읽기
    data_result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', manifest_path
    ], capture_output=True, text=True)
    
    if data_result.returncode == 0:
        lines = data_result.stdout.strip().split('\n')
        
        print(f"\n📊 총 {len(lines)}개 엔트리 발견")
        print(f"🔍 첫 번째 엔트리 상세 분석:")
        print("-" * 50)
        
        # 첫 번째 레코드만 상세 분석
        if lines and lines[0].strip():
            try:
                record = json.loads(lines[0])
                
                # 최상위 필드들
                print("📋 MANIFEST ENTRY 구조:")
                print("=" * 30)
                
                for key, value in record.items():
                    if key == 'data_file' and isinstance(value, dict):
                        print(f"🔹 {key}: [데이터 파일 정보]")
                        print("  📄 DATA FILE 상세:")
                        
                        # data_file 내용을 깔끔하게
                        df = value
                        important_fields = [
                            ('file_path', '파일 경로'),
                            ('file_format', '파일 형식'), 
                            ('record_count', '레코드 수'),
                            ('file_size_in_bytes', '파일 크기'),
                            ('column_sizes', '컬럼별 크기'),
                            ('value_counts', '컬럼별 값 개수'),
                            ('null_value_counts', '컬럼별 NULL 개수'),
                            ('lower_bounds', '컬럼 최솟값'),
                            ('upper_bounds', '컬럼 최댓값')
                        ]
                        
                        for field, description in important_fields:
                            if field in df:
                                val = df[field]
                                if field in ['lower_bounds', 'upper_bounds'] and isinstance(val, dict):
                                    print(f"    🔸 {description}:")
                                    for col_id, bound_val in val.items():
                                        # 바이너리 데이터를 문자열로 시도
                                        try:
                                            if isinstance(bound_val, str):
                                                display_val = bound_val
                                            else:
                                                display_val = str(bound_val)[:50]
                                        except:
                                            display_val = "[바이너리 데이터]"
                                        print(f"      컬럼 {col_id}: {display_val}")
                                elif field in ['column_sizes', 'value_counts', 'null_value_counts'] and isinstance(val, dict):
                                    print(f"    🔸 {description}: {dict(list(val.items())[:3])}{'...' if len(val) > 3 else ''}")
                                else:
                                    if field == 'file_size_in_bytes':
                                        print(f"    🔸 {description}: {val:,} bytes ({val/1024:.1f} KB)")
                                    elif field == 'record_count':
                                        print(f"    🔸 {description}: {val:,}개")
                                    else:
                                        print(f"    🔸 {description}: {val}")
                    else:
                        if key == 'status':
                            status_text = {1: "추가됨", 2: "삭제됨", 0: "기존"}.get(value, f"상태:{value}")
                            print(f"🔹 {key}: {value} ({status_text})")
                        elif key == 'snapshot_id':
                            print(f"🔹 {key}: {value}")
                        else:
                            print(f"🔹 {key}: {value}")
                
            except json.JSONDecodeError as e:
                print(f"❌ JSON 파싱 실패: {e}")
    else:
        print(f"❌ Manifest 읽기 실패: {data_result.stderr}")

# 2. MANIFEST LIST 하나만 상세 분석
print(f"\n\n📋 MANIFEST LIST 상세 분석")
print("=" * 60)

if manifest_lists:
    selected_list = manifest_lists[0]  # 첫 번째 선택
    list_path = os.path.join(metadata_path, selected_list)
    
    print(f"🎯 선택된 파일: {selected_list}")
    print(f"📏 파일 크기: {os.path.getsize(list_path):,} bytes")
    
    # 파일명에서 스냅샷 ID 추출
    try:
        snapshot_id = selected_list.split('-')[1]
        print(f"📸 연관 스냅샷 ID: {snapshot_id}")
    except:
        print(f"📸 스냅샷 ID: 추출 실패")
    
    # avro-tools로 데이터 읽기
    data_result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', list_path
    ], capture_output=True, text=True)
    
    if data_result.returncode == 0:
        lines = data_result.stdout.strip().split('\n')
        
        print(f"\n📊 총 {len(lines)}개 Manifest 참조")
        print(f"🔍 첫 번째 Manifest 참조 상세 분석:")
        print("-" * 50)
        
        # 첫 번째 레코드만 상세 분석
        if lines and lines[0].strip():
            try:
                record = json.loads(lines[0])
                
                print("📋 MANIFEST LIST ENTRY 구조:")
                print("=" * 30)
                
                # 중요한 필드들을 순서대로 정리
                important_fields = [
                    ('manifest_path', '📄 Manifest 파일 경로'),
                    ('manifest_length', '📏 Manifest 파일 크기'),
                    ('partition_spec_id', '🗂️ 파티션 스펙 ID'),
                    ('content', '📊 내용 타입'),
                    ('sequence_number', '🔢 시퀀스 번호'),
                    ('min_sequence_number', '🔢 최소 시퀀스 번호'),
                    ('added_snapshot_id', '📸 추가된 스냅샷 ID'),
                    ('added_data_files_count', '➕ 추가된 데이터 파일 수'),
                    ('existing_data_files_count', '📄 기존 데이터 파일 수'),
                    ('deleted_data_files_count', '🗑️ 삭제된 데이터 파일 수'),
                    ('added_rows_count', '➕ 추가된 레코드 수'),
                    ('existing_rows_count', '📊 기존 레코드 수'),
                    ('deleted_rows_count', '🗑️ 삭제된 레코드 수'),
                ]
                
                for field, description in important_fields:
                    if field in record:
                        value = record[field]
                        
                        if field == 'manifest_path':
                            filename = os.path.basename(value) if value else 'N/A'
                            print(f"🔹 {description}: {filename}")
                            print(f"    전체 경로: {value}")
                        elif field == 'manifest_length':
                            print(f"🔹 {description}: {value:,} bytes ({value/1024:.1f} KB)")
                        elif field == 'content':
                            content_text = {0: "DATA", 1: "DELETE"}.get(value, f"타입:{value}")
                            print(f"🔹 {description}: {value} ({content_text})")
                        elif 'count' in field:
                            print(f"🔹 {description}: {value:,}개")
                        else:
                            print(f"🔹 {description}: {value}")
                
                # 추가 필드들 (있다면)
                other_fields = [k for k in record.keys() if k not in [f[0] for f in important_fields]]
                if other_fields:
                    print(f"\n📋 기타 필드들:")
                    for field in other_fields:
                        value = record[field]
                        if isinstance(value, (dict, list)) and len(str(value)) > 100:
                            print(f"🔹 {field}: [복잡한 객체] ({type(value).__name__})")
                        else:
                            print(f"🔹 {field}: {value}")
                
            except json.JSONDecodeError as e:
                print(f"❌ JSON 파싱 실패: {e}")
    else:
        print(f"❌ Manifest List 읽기 실패: {data_result.stderr}")

# 3. 요약
print(f"\n\n💡 요약")
print("=" * 60)
print("📄 Manifest File (선택됨):")
print(f"  └─ {selected_manifest if manifest_files else 'N/A'}")
print("  └─ 역할: 개별 데이터 파일들의 상세 메타데이터")
print("  └─ 내용: 파일 경로, 크기, 통계, pruning 정보")

print(f"\n📋 Manifest List (선택됨):")
print(f"  └─ {selected_list if manifest_lists else 'N/A'}")
print("  └─ 역할: 스냅샷에 포함된 Manifest 파일들의 목록")
print("  └─ 내용: Manifest 위치, 파일/레코드 변화량")

print(f"\n🔗 관계:")
print("  Snapshot → Manifest List → Manifest Files → Data Files")

📄 MANIFEST FILE 상세 분석
🎯 선택된 파일: 1402a55e-3b81-4e6f-afb1-338c510fc079-m0.avro
📏 파일 크기: 10,910 bytes

📊 총 74개 엔트리 발견
🔍 첫 번째 엔트리 상세 분석:
--------------------------------------------------
📋 MANIFEST ENTRY 구조:
🔹 status: 0 (기존)
🔹 snapshot_id: {'long': 3289915886533077444}
🔹 sequence_number: {'long': 1}
🔹 file_sequence_number: {'long': 1}
🔹 data_file: [데이터 파일 정보]
  📄 DATA FILE 상세:
    🔸 파일 경로: file:/home/jovyan/data/warehouse/blog/orders_mor/data/order_date=2023-01-01/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet
    🔸 파일 형식: PARQUET
    🔸 레코드 수: 1개
    🔸 파일 크기: 1,834 bytes (1.8 KB)
    🔸 컬럼별 크기: {'array': [{'key': 5, 'value': 45}, {'key': 1, 'value': 46}, {'key': 6, 'value': 49}, {'key': 2, 'value': 46}, {'key': 3, 'value': 55}, {'key': 4, 'value': 42}]}
    🔸 컬럼별 값 개수: {'array': [{'key': 5, 'value': 1}, {'key': 1, 'value': 1}, {'key': 6, 'value': 1}, {'key': 2, 'value': 1}, {'key': 3, 'value': 1}, {'key': 4, 'value': 1}]}
    🔸 컬럼별 NULL 개수: {'array': [{'key': 5, 'value': 0}, {'

In [13]:
# =============================================================================
# 🔍 순수 원본 JSON - Pretty Print로 예쁘게
# =============================================================================

import subprocess
import json
import os

metadata_path = f"{mor_table_path}/metadata"
avro_tools_path = "/tmp/avro-tools.jar"

# 파일들 찾기
manifest_files = [f for f in os.listdir(metadata_path) 
                 if f.endswith('.avro') and not f.startswith('snap-')]
manifest_lists = [f for f in os.listdir(metadata_path) 
                  if f.startswith('snap-') and f.endswith('.avro')]

print("📄 MANIFEST FILE 원본 JSON (Pretty Print)")
print("=" * 80)

if manifest_files:
    selected_manifest = manifest_files[0]
    manifest_path = os.path.join(metadata_path, selected_manifest)
    
    print(f"파일: {selected_manifest}")
    print(f"크기: {os.path.getsize(manifest_path):,} bytes")
    print()
    print("원본 JSON (Pretty):")
    print("-" * 80)
    
    # avro-tools 실행
    result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', manifest_path
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        # 첫 번째 레코드를 JSON으로 파싱 후 pretty print
        if lines and lines[0].strip():
            try:
                raw_json = json.loads(lines[0])
                print(json.dumps(raw_json, indent=2, ensure_ascii=False))
            except json.JSONDecodeError:
                print("JSON 파싱 실패, 원본 출력:")
                print(lines[0])
        
        print(f"\n(총 {len(lines)}개 레코드 중 첫 번째만 표시)")
    else:
        print(f"ERROR: {result.stderr}")

print(f"\n\n📋 MANIFEST LIST 원본 JSON (Pretty Print)")
print("=" * 80)

if manifest_lists:
    selected_list = manifest_lists[0]
    list_path = os.path.join(metadata_path, selected_list)
    
    print(f"파일: {selected_list}")
    print(f"크기: {os.path.getsize(list_path):,} bytes")
    print()
    print("원본 JSON (Pretty):")
    print("-" * 80)
    
    # avro-tools 실행
    result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', list_path
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        # 첫 번째 레코드를 JSON으로 파싱 후 pretty print
        if lines and lines[0].strip():
            try:
                raw_json = json.loads(lines[0])
                print(json.dumps(raw_json, indent=2, ensure_ascii=False))
            except json.JSONDecodeError:
                print("JSON 파싱 실패, 원본 출력:")
                print(lines[0])
        
        print(f"\n(총 {len(lines)}개 레코드 중 첫 번째만 표시)")
    else:
        print(f"ERROR: {result.stderr}")

📄 MANIFEST FILE 원본 JSON (Pretty Print)
파일: 1402a55e-3b81-4e6f-afb1-338c510fc079-m0.avro
크기: 10,910 bytes

원본 JSON (Pretty):
--------------------------------------------------------------------------------
{
  "status": 0,
  "snapshot_id": {
    "long": 3289915886533077444
  },
  "sequence_number": {
    "long": 1
  },
  "file_sequence_number": {
    "long": 1
  },
  "data_file": {
    "content": 0,
    "file_path": "file:/home/jovyan/data/warehouse/blog/orders_mor/data/order_date=2023-01-01/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet",
    "file_format": "PARQUET",
    "partition": {
      "order_date": {
        "int": 19358
      }
    },
    "record_count": 1,
    "file_size_in_bytes": 1834,
    "column_sizes": {
      "array": [
        {
          "key": 5,
          "value": 45
        },
        {
          "key": 1,
          "value": 46
        },
        {
          "key": 6,
          "value": 49
        },
        {
          "key": 2,
          "value": 46
 

In [16]:
# 모든 레코드를 pretty print로 보고 싶다면:
print(f"\n\n📄 MANIFEST FILE - 모든 레코드 Pretty Print")
print("=" * 80)

if manifest_files:
    selected_manifest = manifest_files[0]
    print(selected_manifest)
    manifest_path = os.path.join(metadata_path, selected_manifest)
    
    result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', manifest_path
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        print(f"총 {len(lines)}개 레코드:")
        print("-" * 50)
        
        for i, line in enumerate(lines):
            if line.strip():
                try:
                    raw_json = json.loads(line)
                    print(f"\n📝 레코드 #{i+1}:")
                    print(json.dumps(raw_json, indent=2, ensure_ascii=False))
                except json.JSONDecodeError:
                    print(f"\n📝 레코드 #{i+1} (JSON 파싱 실패):")
                    print(line)
    else:
        print(f"ERROR: {result.stderr}")

print(f"\n\n📋 MANIFEST LIST - 모든 레코드 Pretty Print")
print("=" * 80)

if manifest_lists:
    selected_list = manifest_lists[0]
    print(selected_list)
    list_path = os.path.join(metadata_path, selected_list)
    
    result = subprocess.run([
        'java', '-jar', avro_tools_path, 'tojson', list_path
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        print(f"총 {len(lines)}개 레코드:")
        print("-" * 50)
        
        for i, line in enumerate(lines):
            if line.strip():
                try:
                    raw_json = json.loads(line)
                    print(f"\n📝 레코드 #{i+1}:")
                    print(json.dumps(raw_json, indent=2, ensure_ascii=False))
                except json.JSONDecodeError:
                    print(f"\n📝 레코드 #{i+1} (JSON 파싱 실패):")
                    print(line)
    else:
        print(f"ERROR: {result.stderr}")

print(f"\n✨ Pretty Print 완료")



📄 MANIFEST FILE - 모든 레코드 Pretty Print
1402a55e-3b81-4e6f-afb1-338c510fc079-m0.avro
총 74개 레코드:
--------------------------------------------------

📝 레코드 #1:
{
  "status": 0,
  "snapshot_id": {
    "long": 3289915886533077444
  },
  "sequence_number": {
    "long": 1
  },
  "file_sequence_number": {
    "long": 1
  },
  "data_file": {
    "content": 0,
    "file_path": "file:/home/jovyan/data/warehouse/blog/orders_mor/data/order_date=2023-01-01/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet",
    "file_format": "PARQUET",
    "partition": {
      "order_date": {
        "int": 19358
      }
    },
    "record_count": 1,
    "file_size_in_bytes": 1834,
    "column_sizes": {
      "array": [
        {
          "key": 5,
          "value": 45
        },
        {
          "key": 1,
          "value": 46
        },
        {
          "key": 6,
          "value": 49
        },
        {
          "key": 2,
          "value": 46
        },
        {
          "key": 3,
       